In [1]:
import numpy as np
import pandas as pd
from scipy.stats import kstest

Generate surfaces before...

In [2]:
num_epochs = 500
ctx_len = 4
start_day = 4950
days_to_generate = 100
num_vaes = 1000

data = np.load("data/vol_surface_with_ret.npz")
vol_surf_data = data["surface"]
ret_data = data["ret"]
level_data = data["levels"]
skew_data = data["skews"]
slope_data = data["slopes"]

# no training on ex feats

In [3]:
# default is 2 sided The null hypothesis is that the two distributions are identical, F(x)=G(x) for all x; the alternative is that they are not identical. We should expect a p_value >= 0.05
# two-sample test, statistic_location is the value from rvs or cdf corresponding with the KS statistic; i.e., the distance between the empirical distribution functions is measured at this observation.
# statistic sign: +1 if the empirical distribution function of rvs exceeds the empirical distribution function of cdf at statistic_location, otherwise -1.
result_df = {
    "model_type": [],
    "ctx_len": [],
    "ks_stats": [],
    "p_value": [],
    "statistic_location": [],
    "statistic_sign": [],
}
real_surface_levels = level_data[start_day:start_day+days_to_generate]
for model_type in ["conv_mem_spx_ex_rand_compress_context_0",
                   "conv_mem_spx_ex3_0",
                   "conv_mem_spx_rand_compress_context_0"]:
    for ctx_len in [2, 3, 4, 5]:
        generated = np.load(f"test_spx/simpath/{model_type}_gen{ctx_len}.npy") #(num_day, num_sample, H, W)
        generated_levels = generated[:, :, 3, 2].reshape((-1,))
        ks_test_result = kstest(generated_levels, real_surface_levels)
        result_df["model_type"].append(model_type)
        result_df["ctx_len"].append(ctx_len)
        result_df["ks_stats"].append(ks_test_result.statistic)
        result_df["p_value"].append(ks_test_result.pvalue)
        result_df["statistic_location"].append(ks_test_result.statistic_location)
        result_df["statistic_sign"].append(ks_test_result.statistic_sign)
result_df = pd.DataFrame(result_df)
result_df.to_csv("test_spx/ks_stats1.csv")

In [5]:
# default is 2 sided The null hypothesis is that the two distributions are identical, F(x)=G(x) for all x; the alternative is that they are not identical. We should expect a p_value >= 0.05
# two-sample test, statistic_location is the value from rvs or cdf corresponding with the KS statistic; i.e., the distance between the empirical distribution functions is measured at this observation.
# statistic sign: +1 if the empirical distribution function of rvs exceeds the empirical distribution function of cdf at statistic_location, otherwise -1.

# null hypothesis mean: the distribution of the mean value should be identical (similar) to the realized
# null hypothesis std: the distribution of the std should be identical to constant
result_df = {
    "model_type": [],
    "ctx_len": [],
    "mean_ks_stats": [],
    "mean_p_value": [],
    "mean_statistic_location": [],
    "mean_statistic_sign": [],
    "std_ks_stats": [],
    "std_p_value": [],
    "std_statistic_location": [],
    "std_statistic_sign": [],
}
real_surface_levels = level_data[start_day:start_day+days_to_generate]
for model_type in ["conv_mem_spx_ex_rand_compress_context_0",
                   "conv_mem_spx_ex3_0",
                   "conv_mem_spx_rand_compress_context_0"]:
    for ctx_len in [2, 3, 4, 5]:
        generated = np.load(f"test_spx/simpath/{model_type}_gen{ctx_len}.npy") #(num_day, num_sample, H, W)
        generated_levels = generated[:, :, 3, 2]
        ks_test_result = kstest(np.mean(generated_levels, axis=1), real_surface_levels)
        result_df["model_type"].append(model_type)
        result_df["ctx_len"].append(ctx_len)
        result_df["mean_ks_stats"].append(ks_test_result.statistic)
        result_df["mean_p_value"].append(ks_test_result.pvalue)
        result_df["mean_statistic_location"].append(ks_test_result.statistic_location)
        result_df["mean_statistic_sign"].append(ks_test_result.statistic_sign)
        ks_test_result = kstest(np.std(generated_levels, axis=1), np.ones_like(real_surface_levels))
        result_df["std_ks_stats"].append(ks_test_result.statistic)
        result_df["std_p_value"].append(ks_test_result.pvalue)
        result_df["std_statistic_location"].append(ks_test_result.statistic_location)
        result_df["std_statistic_sign"].append(ks_test_result.statistic_sign)
result_df = pd.DataFrame(result_df)
result_df.to_csv("test_spx/ks_stats1-2.csv")

# trained on ex feats

In [4]:
result_df = {
    "model_type": [],
    "ctx_len": [],
    "level_ks_stats": [],
    "level_p_value": [],
    "level_statistic_location": [],
    "level_statistic_sign": [],
    "ret_ks_stats": [],
    "ret_p_value": [],
    "ret_statistic_location": [],
    "ret_statistic_sign": [],
}
real_surface_levels = level_data[start_day:start_day+days_to_generate]
real_ret = ret_data[start_day:start_day+days_to_generate]
for model_type in ["conv_mem_spx_ex_rand_compress_context_1",
                   "conv_mem_spx_ex3_1"]:
    for ctx_len in [2, 3, 4, 5]:
        generated = np.load(f"test_spx/simpath/{model_type}_gen{ctx_len}.npz")
        surfaces = generated["surfaces"] #(num_day, num_sample, H, W)
        ex_feats = generated["ex_feats"] #(num_day, num_sample, n_ex_feats)
        generated_levels = surfaces[:, :, 3, 2].reshape((-1,))
        generated_rets = ex_feats[:, :, 0].reshape((-1,))
        ks_test_result = kstest(generated_levels, real_surface_levels)
        ks_test_result2 = kstest(generated_rets, real_ret)
        result_df["model_type"].append(model_type)
        result_df["ctx_len"].append(ctx_len)
        result_df["level_ks_stats"].append(ks_test_result.statistic)
        result_df["level_p_value"].append(ks_test_result.pvalue)
        result_df["level_statistic_location"].append(ks_test_result.statistic_location)
        result_df["level_statistic_sign"].append(ks_test_result.statistic_sign)
        result_df["ret_ks_stats"].append(ks_test_result2.statistic)
        result_df["ret_p_value"].append(ks_test_result2.pvalue)
        result_df["ret_statistic_location"].append(ks_test_result2.statistic_location)
        result_df["ret_statistic_sign"].append(ks_test_result2.statistic_sign)
result_df = pd.DataFrame(result_df)
result_df.to_csv("test_spx/ks_stats2.csv")

In [6]:
# null hypothesis mean: the distribution of the mean value should be identical (similar) to the realized
# null hypothesis std: the distribution of the std should be identical to constant
result_df = {
    "model_type": [],
    "ctx_len": [],
    "level_mean_ks_stats": [],
    "level_mean_p_value": [],
    "level_mean_statistic_location": [],
    "level_mean_statistic_sign": [],
    "level_std_ks_stats": [],
    "level_std_p_value": [],
    "level_std_statistic_location": [],
    "level_std_statistic_sign": [],
    "ret_mean_ks_stats": [],
    "ret_mean_p_value": [],
    "ret_mean_statistic_location": [],
    "ret_mean_statistic_sign": [],
    "ret_std_ks_stats": [],
    "ret_std_p_value": [],
    "ret_std_statistic_location": [],
    "ret_std_statistic_sign": [],
}
real_surface_levels = level_data[start_day:start_day+days_to_generate]
real_ret = ret_data[start_day:start_day+days_to_generate]
for model_type in ["conv_mem_spx_ex_rand_compress_context_1",
                   "conv_mem_spx_ex3_1"]:
    for ctx_len in [2, 3, 4, 5]:
        generated = np.load(f"test_spx/simpath/{model_type}_gen{ctx_len}.npz")
        surfaces = generated["surfaces"] #(num_day, num_sample, H, W)
        ex_feats = generated["ex_feats"] #(num_day, num_sample, n_ex_feats)
        generated_levels = surfaces[:, :, 3, 2]
        generated_rets = ex_feats[:, :, 0]
        ks_test_result = kstest(np.mean(generated_levels, axis=1), real_surface_levels)
        ks_test_result2 = kstest(np.mean(generated_rets, axis=1), real_ret)
        result_df["model_type"].append(model_type)
        result_df["ctx_len"].append(ctx_len)
        result_df["level_mean_ks_stats"].append(ks_test_result.statistic)
        result_df["level_mean_p_value"].append(ks_test_result.pvalue)
        result_df["level_mean_statistic_location"].append(ks_test_result.statistic_location)
        result_df["level_mean_statistic_sign"].append(ks_test_result.statistic_sign)
        result_df["ret_mean_ks_stats"].append(ks_test_result2.statistic)
        result_df["ret_mean_p_value"].append(ks_test_result2.pvalue)
        result_df["ret_mean_statistic_location"].append(ks_test_result2.statistic_location)
        result_df["ret_mean_statistic_sign"].append(ks_test_result2.statistic_sign)

        ks_test_result = kstest(np.std(generated_levels, axis=1), np.ones_like(real_surface_levels))
        ks_test_result2 = kstest(np.std(generated_rets, axis=1), np.ones_like(real_surface_levels))
        result_df["level_std_ks_stats"].append(ks_test_result.statistic)
        result_df["level_std_p_value"].append(ks_test_result.pvalue)
        result_df["level_std_statistic_location"].append(ks_test_result.statistic_location)
        result_df["level_std_statistic_sign"].append(ks_test_result.statistic_sign)
        result_df["ret_std_ks_stats"].append(ks_test_result2.statistic)
        result_df["ret_std_p_value"].append(ks_test_result2.pvalue)
        result_df["ret_std_statistic_location"].append(ks_test_result2.statistic_location)
        result_df["ret_std_statistic_sign"].append(ks_test_result2.statistic_sign)
result_df = pd.DataFrame(result_df)
result_df.to_csv("test_spx/ks_stats2-2.csv")